<a href="https://colab.research.google.com/github/Kirknasty54/Jackson_Kirkpatrick_ICP9/blob/main/Jackson_Kirkpatrick_ICP_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install scikeras
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, load_model
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import re
from sklearn.preprocessing import LabelEncoder
from google.colab import drive
drive.mount('/content/drive')
data = pd.read_csv('/content/drive/MyDrive/Sentiment.csv')
import joblib
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.pipeline import Pipeline

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Keeping only the neccessary columns
data = data[['text','sentiment']]

data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))

for idx, row in data.iterrows():
    row[0] = row[0].replace('rt', ' ')

max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)

X = pad_sequences(X)

embed_dim = 128
lstm_out = 196
def createmodel(embed_dim, lstm_out):
    model = Sequential()
    model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(3,activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
    return model
print(createmodel(embed_dim, lstm_out).summary())

labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(data['sentiment'])
y = to_categorical(integer_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.33, random_state = 42)

batch_size = 32
model = createmodel(embed_dim, lstm_out)
model.fit(X_train, Y_train, epochs = 1, batch_size=batch_size, verbose = 2)
score,acc = model.evaluate(X_test,Y_test,verbose=2,batch_size=batch_size)
print(score)
print(acc)
print(model.metrics_names)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 28, 128)           256000    
                                                                 
 lstm (LSTM)                 (None, 196)               254800    
                                                                 
 dense (Dense)               (None, 3)                 591       
                                                                 
Total params: 511391 (1.95 MB)
Trainable params: 511391 (1.95 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
291/291 - 74s - loss: 0.8250 - accuracy: 0.6460 - 74s/epoch - 254ms/step
144/144 - 4s - loss: 0.7546 - accuracy: 0.6763 - 4s/epoch - 25ms/step
0.7546238899230957
0.6762778759002686
['loss', 'accuracy']


In [3]:
model.save("sentiment_analysis_model.h5")
loaded_model = load_model("sentiment_analysis_model.h5")

#create a new text data for predictions
new_tweet = ["A lot of good things are happening. We are respected again throughout the world, and that's a great thing. @realDonaldTrump"]

#preprocess our new text data the same way we did for our training data
new_tweet = [x.lower() for x in new_tweet]
new_tweet = [re.sub('[^a-zA-z0-9\s]', '', x) for x in new_tweet]

new_tweet_sequences = tokenizer.texts_to_sequences(new_tweet)
new_tweet_sequences = pad_sequences(new_tweet_sequences, maxlen=X.shape[1])

predictions = loaded_model.predict(new_tweet_sequences)
predicted_sentiments = labelencoder.inverse_transform(predictions.argmax(axis=1))
print(predicted_sentiments)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1/1 [==============================] - 1s 855ms/step
['Positive']


In [5]:
'''# Load the Iris dataset
irisd = load_iris()
X = irisd.data
y = irisd.target

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define a function to create the Keras model
def create_model(units=10, activation='relu'):
   model = Sequential()
   model.add(Dense(units=units, activation=activation, input_dim=4))
   model.add(Dense(units=3, activation='softmax'))
   model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
   return model

# Create the KerasClassifier object
model = KerasClassifier(build_fn=create_model, activation='relu', units=5)

# Define the hyperparameter grid to search over
param_grid = {
   'units': [5, 10, 15],
   'activation': ['relu', 'sigmoid']
}

# Create the GridSearchCV object
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)

# Fit the GridSearchCV object to the training data
grid_result = grid.fit(X_train, y_train)

# Print the best parameters and score
print("Best Parameters: ", grid_result.best_params_)
print("Best Score: ", grid_result.best_score_)

# Evaluate the best model on the test data
best_model = grid_result.best_estimator_
test_accuracy = best_model.score(X_test, y_test)
print("Test Accuracy: ", test_accuracy)'''
model = KerasClassifier(build_fn=createmodel, epochs=1, batch_size=16, lstm_out=64, embed_dim=64)

# Define the hyperparameter grid for GridSearchCV
param_grid = {
    'embed_dim': [64, 128],
    'lstm_out': [64, 128],
    'batch_size': [16, 32]
}

# Create GridSearchCV
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, error_score='raise')

# Fit GridSearchCV to the data
grid_result = grid.fit(X, y)

# Print the best parameters and the corresponding best cross-validated score
print("Best Parameters: ", grid_result.best_params_)
print("Best Score: ", grid_result.best_score_)

# Create a new model with the best hyperparameters and evaluate it on the test data
best_params = grid_result.best_params_
best_model = createmodel(embed_dim=best_params['embed_dim'], lstm_out=best_params['lstm_out'])
best_model.fit(X, y, epochs=1, batch_size=batch_size, verbose=2)
score, acc = best_model.evaluate(X_test, Y_test, verbose=2, batch_size=batch_size)
print("Test Score: ", score)
print("Test Accuracy: ", acc)

/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


289/289 [==============================] - 2s 7ms/step


/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


289/289 [==============================] - 3s 9ms/step


/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


...

145/145 [==============================] - 3s 16ms/step


/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


145/145 [==============================] - 4s 23ms/step


/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


867/867 [==============================] - 64s 69ms/step - loss: 0.7853 - accuracy: 0.6625
Best Parameters:  {'batch_size': 16, 'embed_dim': 128, 'lstm_out': 64}
Best Score:  0.6592190899721343
434/434 - 40s - loss: 0.7965 - accuracy: 0.6594 - 40s/epoch - 92ms/step
144/144 - 3s - loss: 0.6482 - accuracy: 0.7204 - 3s/epoch - 22ms/step
Test Score:  0.6481953859329224
Test Accuracy:  0.72040194272995
